In [10]:
import kimports; reload(kimports); from kimports import *
import kutils; reload(kutils)
from keras.utils.data_utils import get_file

In [12]:
from keras.datasets import imdb
import pickle

In [3]:
word2idx = imdb.get_word_index()
idx2word = {idx: word for word, idx in word2idx.iteritems()}

In [17]:
# this data is all screwed up, the reviews dont make sense
#(x_train, y_train), (x_test, y_test) = imdb.load_data()

In [13]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [16]:
' '.join(map(lambda x: idx2word[x], x_train[0]))

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [47]:
vocab_size=5000
max_len=500
num_factors = 32

In [40]:
train_data = [np.array([i if i < vocab_size else vocab_size - 1 for i in review]) for review in x_train]
train_data = keras.preprocessing.sequence.pad_sequences(train_data, maxlen=max_len)
train_labels = labels_train
valid_data = [np.array([i if i < vocab_size else vocab_size - 1 for i in review]) for review in x_test]
valid_data = keras.preprocessing.sequence.pad_sequences(valid_data, maxlen=max_len)
valid_labels = labels_test

In [41]:
train_data.shape

(25000, 500)

# Setup model with embedding

In [66]:
review_input = keras.layers.Input(shape=(max_len,))
review_embedding = keras.layers.embeddings.Embedding(vocab_size, num_factors, input_length=max_len, W_regularizer=keras.regularizers.l2(0.001))(review_input)
flattened = keras.layers.core.Flatten()(review_embedding)
dense = keras.layers.core.Dense(100, activation='relu')(flattened)
dropout = keras.layers.core.Dropout(0.7)(dense)
result = keras.layers.core.Dense(1, activation='sigmoid')(dropout)

In [69]:
model = keras.models.Model(review_input, result)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [70]:
model.fit(train_data, train_labels,
          validation_data=(valid_data, valid_labels),
          nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 4s - loss: 0.4128 - acc: 0.8423 - val_loss: 0.3317 - val_acc: 0.8540
Epoch 2/2
25000/25000 [==============================] - 4s - loss: 0.3002 - acc: 0.9103 - val_loss: 0.3250 - val_acc: 0.8632


# With some convolutions

In [114]:
review_input = keras.layers.Input(shape=(max_len,))
review_embedding = keras.layers.embeddings.Embedding(vocab_size, num_factors, input_length=max_len, W_regularizer=keras.regularizers.l2(0.001))(review_input)
conv1 = keras.layers.convolutional.Convolution1D(64, 5, activation='relu')(review_embedding)
pooled = keras.layers.pooling.MaxPooling1D()(conv1)
conv2 = keras.layers.convolutional.Convolution1D(64, 5, activation='relu')(pooled)
flattened = keras.layers.core.Flatten()(conv2)
dense = keras.layers.core.Dense(100, activation='relu')(flattened)
dropout = keras.layers.core.Dropout(0.7)(dense)
result = keras.layers.core.Dense(1, activation='sigmoid')(dropout)
model = keras.models.Model(review_input, result)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [115]:
model.fit(train_data, train_labels,
          validation_data=(valid_data, valid_labels),
          nb_epoch=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 14s - loss: 0.4830 - acc: 0.7671 - val_loss: 0.2801 - val_acc: 0.8849


In [116]:
model.optimizer.lr=0.01

In [117]:
model.fit(train_data, train_labels,
          validation_data=(valid_data, valid_labels),
          nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 14s - loss: 0.3221 - acc: 0.8982 - val_loss: 0.2562 - val_acc: 0.8918
Epoch 2/4
25000/25000 [==============================] - 14s - loss: 0.2960 - acc: 0.9100 - val_loss: 0.2541 - val_acc: 0.8942
Epoch 3/4
25000/25000 [==============================] - 14s - loss: 0.2675 - acc: 0.9237 - val_loss: 0.2602 - val_acc: 0.8920
Epoch 4/4
25000/25000 [==============================] - 14s - loss: 0.2532 - acc: 0.9328 - val_loss: 0.2696 - val_acc: 0.8938


# Load GloVe

In [173]:
num_factors = 50

In [174]:
glove_vectors = {}
with open('../data/glove/glove.6B.50d.txt') as f:
    for l in f:
        tokens = l.split()
        glove_vectors[tokens[0]] = np.array([map(float, tokens[1:])])

In [188]:
emb = np.zeros(((vocab_size, num_factors)))
for idx in sorted(idx2word.keys())[:vocab_size-1]:
    try:
        emb[idx] = glove_vectors[idx2word[idx]]
    except KeyError:
        # word mismatch between glove and imdb words
        emb[idx] = np.random.rand(num_factors)
        
# Set the "everything else" word to random
emb[-1] = np.random.rand(num_factors)

# Conv model with GloVe

In [193]:
review_input = keras.layers.Input(shape=(max_len,))
review_embedding = keras.layers.embeddings.Embedding(vocab_size, num_factors, input_length=max_len, weights=[emb])(review_input)
conv1 = keras.layers.convolutional.Convolution1D(64, 5, activation='relu')(review_embedding)
pooled = keras.layers.pooling.MaxPooling1D()(conv1)
conv2 = keras.layers.convolutional.Convolution1D(64, 5, activation='relu')(pooled)
flattened = keras.layers.core.Flatten()(conv2)
dense = keras.layers.core.Dense(100, activation='relu')(flattened)
dropout = keras.layers.core.Dropout(0.7)(dense)
result = keras.layers.core.Dense(1, activation='sigmoid')(dropout)
model = keras.models.Model(review_input, result)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [194]:
model.fit(train_data, train_labels,
          validation_data=(valid_data, valid_labels),
          nb_epoch=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 15s - loss: 0.6941 - acc: 0.5039 - val_loss: 0.6920 - val_acc: 0.5244


In [195]:
model.optimizer.lr = 0.01

In [196]:
model.fit(train_data, train_labels,
          validation_data=(valid_data, valid_labels),
          nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 15s - loss: 0.5516 - acc: 0.7013 - val_loss: 0.3163 - val_acc: 0.8639
Epoch 2/4
25000/25000 [==============================] - 15s - loss: 0.2950 - acc: 0.8836 - val_loss: 0.2837 - val_acc: 0.8774
Epoch 3/4
25000/25000 [==============================] - 15s - loss: 0.2286 - acc: 0.9150 - val_loss: 0.2527 - val_acc: 0.8973
Epoch 4/4
25000/25000 [==============================] - 15s - loss: 0.1838 - acc: 0.9306 - val_loss: 0.2830 - val_acc: 0.8878
